In [1]:
from bread.vis import *
from bread.data import Features, SegmentationFile, Microscopy
from bread.algo.lineage import LineageGuesser, LineageGuesserNN, LineageGuesserML, LineageGuesserNearestCell, accuracy
from bread.data import Lineage

import pandas as pd
import pathlib
import os

# test for colony 0 to 5 individually


In [2]:
args = {'fov': 0, 'bud_distance_max': 12, 'num_frames_refractory': 0, 'num_frames': 8}
data_path = '../../../data/'
model_path = '/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/saved_models/best_model_with_fake_candid_thresh12_frame_num8_normalized_False.pth'
lineage_guesses = []

# find predictions for each colony
for colony in range(0,6):
    output_file = os.path.join(data_path, f'colony00{colony}_lineage_guess.csv')
    segmentation_file = os.path.join(data_path, f'colony00{colony}_segmentation.h5')
    segmentation = SegmentationFile.from_h5(segmentation_file).get_segmentation('FOV'+str(args['fov']))
    guesser = LineageGuesserNN(
        saved_model=model_path,
        segmentation=segmentation,
        dist_threshold=args['bud_distance_max'],
        num_frames_refractory=args['num_frames_refractory'],
        num_frames=args['num_frames'],
    )

    lineage_guess: Lineage = guesser.guess_lineage()
    lineage_guesses.append(lineage_guess)
    lineage_guess.save_csv(output_file)

/home/farzaneh/Documents/Bread/bread/src/bread/data/_data.py:596: MultipleContoursWarning: OpenCV returned multiple contours, 2 found.
  warnings.warn(Contour.MultipleContoursWarning(len(contours_cv)))
/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/_lineage.py:219: BreadWarning: cell #44 does not have nearest neighbours with a distance less than 12, and flexible_threshold is False.
  warnings.warn(BreadWarning(f'cell #{nearest_neighbours_of} does not have nearest neighbours with a distance less than {self.dist_threshold}, and flexible_threshold is {self.flexible_nn_threshold}.'))
/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/_lineage.py:141: NoGuessWarning: Unable to determine parent for bud #185 in frame #122. Got error CellMissingException('Unable to find cell_id=155 at time_id=124 in the segmentation.')
  warnings.warn(LineageGuesser.NoGuessWarning(bud_id, time_id, e))
/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/_lineage.py:953: NotEnoughFr

In [3]:
# find accuracy for each colony
for colony, lin_guess in enumerate(lineage_guesses):
    lineage_gt_path = os.path.join(data_path, f'colony00{colony}_lineage_gt.csv')
    lin_truth = Lineage.from_csv(lineage_gt_path)
    acc = accuracy(lin_truth, lin_guess, strict=False)
    print('accuracy for colony {}: {}'.format(colony , acc))
    

accuracy for colony 0: 0.8652037617554859
accuracy for colony 1: 0.9315068493150684
accuracy for colony 2: 0.8863636363636364
accuracy for colony 3: 0.8414634146341463
accuracy for colony 4: 0.8317757009345794
accuracy for colony 5: 0.9090909090909091
